In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
train = pd.read_csv('../data/train_featureV1.csv')
test = pd.read_csv('../data/test_featureV1.csv')

In [14]:
train.head()

,uid,label,voice_opp_num_unique_count,voice_opp_num_count,voice_opp_head_unique_count,voice_opp_len_3,voice_opp_len_5,voice_opp_len_6,voice_opp_len_7,voice_opp_len_8,...,"('w_a_downflow_max', 0.0)","('w_a_downflow_max', 1.0)","('w_a_downflow_min', 0.0)","('w_a_downflow_min', 1.0)","('w_a_downflow_median', 0.0)","('w_a_downflow_median', 1.0)","('w_a_downflow_mean', 0.0)","('w_a_downflow_mean', 1.0)","('w_a_downflow_sum', 0.0)","('w_a_downflow_sum', 1.0)"
0,u0001,0,22.0,79.0,17.0,0.0,3.0,0.0,0.0,0.0,...,32670956.0,58773144.0,0.0,0.0,9431.0,13088.0,5.127058e+05,2.359752e+06,4.896340e+08,4.719503e+08
1,u0002,0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,15438.0,0.0,0.0,0.0,1656.0,0.0,3.267207e+03,0.000000e+00,9.474900e+04,0.000000e+00
2,u0003,0,15.0,21.0,10.0,0.0,1.0,0.0,0.0,2.0,...,39722861.0,457021859.0,0.0,0.0,5389.5,8130.0,2.360312e+05,5.786216e+06,2.317827e+08,1.070450e+09
3,u0004,0,77.0,254.0,31.0,0.0,1.0,0.0,0.0,12.0,...,513377344.0,94557753.0,0.0,0.0,11194.5,266204.5,1.487322e+06,4.732882e+06,2.040605e+09,1.457728e+09
4,u0005,0,55.0,401.0,28.0,0.0,4.0,0.0,0.0,0.0,...,28858654.0,15441021.0,0.0,0.0,7079.0,5008.5,2.864763e+05,8.140799e+05,1.595673e+08,6.838271e+07


In [15]:
dtrain = lgb.Dataset(train.drop(['uid','label'],axis=1),label=train.label)
dtest = lgb.Dataset(test.drop(['uid'],axis=1))

In [24]:
lgb_params =  {
    'boosting_type': 'gbdt',
    'objective': 'binary',
#    'metric': ('multi_logloss', 'multi_error'),
    #'metric_freq': 100,
    'is_training_metric': False,
    'min_data_in_leaf': 10,
    'num_leaves': 80,
    'learning_rate': 0.08,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'verbosity':-1,
    'max_depth':7,
    'colsample_bytree':0.7,
#    'gpu_device_id':2,
#    'device':'gpu'
#    'lambda_l1': 0.001,
#    'skip_drop': 0.95,
#    'max_drop' : 10
    #'lambda_l2': 0.005
    #'num_threads': 18
}    

In [25]:
def evalMetric(preds,dtrain):
    
    label = dtrain.get_label()
    
    
    pre = pd.DataFrame({'preds':preds,'label':label})
    pre= pre.sort_values(by='preds',ascending=False)
    
    auc = metrics.roc_auc_score(pre.label,pre.preds)

    pre.preds=pre.preds.map(lambda x: 1 if x>=0.38 else 0)

    f1 = metrics.f1_score(pre.label,pre.preds)
    
    
    res = 0.6*auc +0.4*f1
    
    return 'res',res,True
    

    

### 本地CV

In [26]:
lgb.cv(lgb_params,dtrain,feval=evalMetric,early_stopping_rounds=100,verbose_eval=5,num_boost_round=10000,nfold=3,metrics=['evalMetric'])

[5]	cv_agg's res: 0.759932 + 0.0119759
[10]	cv_agg's res: 0.79407 + 0.0121382
[15]	cv_agg's res: 0.794452 + 0.0150241
[20]	cv_agg's res: 0.795515 + 0.0125295
[25]	cv_agg's res: 0.795087 + 0.012113
[30]	cv_agg's res: 0.790841 + 0.0143583
[35]	cv_agg's res: 0.789017 + 0.0126643
[40]	cv_agg's res: 0.790682 + 0.0139865
[45]	cv_agg's res: 0.791282 + 0.0132608
[50]	cv_agg's res: 0.791074 + 0.0137795
[55]	cv_agg's res: 0.793288 + 0.0141672
[60]	cv_agg's res: 0.792165 + 0.0131735
[65]	cv_agg's res: 0.792153 + 0.0157072
[70]	cv_agg's res: 0.790359 + 0.0137162
[75]	cv_agg's res: 0.789939 + 0.014112
[80]	cv_agg's res: 0.791081 + 0.0148956
[85]	cv_agg's res: 0.790022 + 0.0147647
[90]	cv_agg's res: 0.79116 + 0.0152865
[95]	cv_agg's res: 0.788404 + 0.0147141
[100]	cv_agg's res: 0.786947 + 0.0152878
[105]	cv_agg's res: 0.786729 + 0.0135621
[110]	cv_agg's res: 0.786732 + 0.0121489
[115]	cv_agg's res: 0.786346 + 0.0114264
[120]	cv_agg's res: 0.7849 + 0.0133277


{'res-mean': [0.6189202059660609,
  0.6334517970182021,
  0.6490616623761855,
  0.7366436372193085,
  0.7599322341980687,
  0.776879593451124,
  0.7865206121599039,
  0.7888339659038484,
  0.7903066344689155,
  0.7940697922683895,
  0.7897062061222117,
  0.7898230268709848,
  0.7919881870066491,
  0.7943071846873329,
  0.794451629909425,
  0.7947962189063021,
  0.7935582167789796,
  0.7954641103250669,
  0.7960830143395139,
  0.7955147913961037,
  0.7947193061745502,
  0.7956679686001603,
  0.7959673758635718,
  0.796776981364756],
 'res-stdv': [0.010984298351284984,
  0.008493052818745671,
  0.006427488105890213,
  0.006738530864902069,
  0.01197594860512147,
  0.013430802644702254,
  0.012729052361489818,
  0.010836004459353227,
  0.011312145345685285,
  0.012138168012505064,
  0.011927984182375635,
  0.012503794090851368,
  0.01626993343836069,
  0.01641455629198395,
  0.015024100078308468,
  0.01539950004631427,
  0.01288821280018742,
  0.01162247211151992,
  0.012027188410682569,


## 训练

In [19]:
model =lgb.train(lgb_params,dtrain,feval=evalMetric,verbose_eval=5,num_boost_round=300,valid_sets=[dtrain])

[5]	training's res: 0.833491
[10]	training's res: 0.892536
[15]	training's res: 0.907515
[20]	training's res: 0.922982
[25]	training's res: 0.937815
[30]	training's res: 0.94724
[35]	training's res: 0.954062
[40]	training's res: 0.958963
[45]	training's res: 0.962343
[50]	training's res: 0.966902
[55]	training's res: 0.968123
[60]	training's res: 0.972051
[65]	training's res: 0.973827
[70]	training's res: 0.976006
[75]	training's res: 0.979741
[80]	training's res: 0.984728
[85]	training's res: 0.987925
[90]	training's res: 0.989498
[95]	training's res: 0.991284
[100]	training's res: 0.991994
[105]	training's res: 0.993129
[110]	training's res: 0.994686
[115]	training's res: 0.995574
[120]	training's res: 0.997126
[125]	training's res: 0.997787
[130]	training's res: 0.998008
[135]	training's res: 0.99867
[140]	training's res: 0.999556
[145]	training's res: 0.999556
[150]	training's res: 0.999556
[155]	training's res: 0.999778
[160]	training's res: 1
[165]	training's res: 1
[170]	trainin

### 预测

In [9]:
pred=model.predict(test.drop(['uid'],axis=1))

In [10]:
res =pd.DataFrame({'uid':test.uid,'label':pred})


In [11]:
res=res.sort_values(by='label',ascending=False)
res.label=res.label.map(lambda x: 1 if x>=0.38 else 0)
#res.label = res.label.map(lambda x: int(x))

In [12]:
res.to_csv('../result/lgb-baseline.csv',index=False,header=False,sep=',',columns=['uid','label'])